In [1]:
!pip install -q langchain transformers accelerate sentencepiece huggingface_hub
!pip install langchain_community
!pip install gradio

In [2]:
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline


In [3]:
from huggingface_hub import login

HUGGINGFACE_TOKEN = "----"  # 🔴 Replace this with your real token
login(HUGGINGFACE_TOKEN)


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

model_id = "openchat/openchat-3.5-0106"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [9]:
# Sassy recipe bot logic
def sassy_recipe_bot(ingredients):
    prompt = f"""<|system|>
You're a bold, sassy robot chef. For any given list of ingredients, you:
1. Lightly roast the user's choices with humor and sarcasm.
2. Suggest a quirky, over-the-top dish.
3. Present the recipe in well-formatted bullet points.
Keep it short, fun, and a little dramatic. Never be boring!

<|user|>
Hey sassy robot chef, I have {ingredients}. What should I make?

<|assistant|>
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=1.2,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the assistant's reply only
    if "<|assistant|>" in decoded:
        response = decoded.split("<|assistant|>")[-1].strip()
    else:
        response = decoded.replace(prompt, "").strip()

    return response.replace("**", "")

# ✅ Gradio UI with forced pink background
with gr.Blocks(css="""
    .main-container {
        background-color: pink !important;
        padding: 20px;
        min-height: 100vh;
    }
    h4 {
        color: black !important;
    }
""") as demo:
    with gr.Column(elem_classes=["main-container"]):
        gr.Markdown("## 🤖🍳 Sassy Robot Chef: Recipe Generator")
        gr.Markdown("<h4>Enter your ingredients below and prepare to be deliciously insulted!</h4>")

        input_box = gr.Textbox(label="Your Ingredients", placeholder="e.g., banana, milk, chocolate")
        output_box = gr.Textbox(label="Sassy Recipe", lines=12)

        btn = gr.Button("Cook Me Something Fun!")
        btn.click(fn=sassy_recipe_bot, inputs=input_box, outputs=output_box)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3a47ba21a3355101f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://3a47ba21a3355101f4.gradio.live


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1fd7068753f45c49d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
